#Szükséges könyvtárak beimportálása

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd

#Adatgyűjtés (webscrapping)

##Adatgyűjtés előkészítése

*   Szükséges könyvtárak beimportálása
*   Csapatok URL címeinek előkészítése és olyan állapotba hozása, hogy a program tudjon navigálni rajtuk
*   Szükséges táblák adatainak beimportálása, azok mergelése






In [ ]:
!pip install requests

In [ ]:
url_stats = "https://fbref.com/en/comps/13/2022-2023/2022-2023-Ligue-1-Stats"

In [ ]:
data = requests.get(url_stats)

In [ ]:
soup = BeautifulSoup(data.text)
BLstats_table = soup.select('table.stats_table')[0]

In [ ]:
#a tag-ek megtalálása (csapatok linkjei)
links = BLstats_table.find_all('a')

In [ ]:
#a tag-ek értékeinek megtalálása
links = [l.get("href") for l in links]

In [ ]:
#csak a csapatok a tageinek megtartása, így csak a csapatok linkjei maradnak meg
links = [l for l in links if '/squads/' in l]

In [ ]:
#linkek eleje lemaradt, így kiegészítődnek
team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_url=team_urls[0]
data=requests.get(team_url)

In [ ]:
team_urls

['https://fbref.com/en/squads/e2d8892c/2022-2023/Paris-Saint-Germain-Stats',
 'https://fbref.com/en/squads/fd4e0f7d/2022-2023/Lens-Stats',
 'https://fbref.com/en/squads/5725cc7b/2022-2023/Marseille-Stats',
 'https://fbref.com/en/squads/b3072e00/2022-2023/Rennes-Stats',
 'https://fbref.com/en/squads/cb188c0c/2022-2023/Lille-Stats',
 'https://fbref.com/en/squads/fd6114db/2022-2023/Monaco-Stats',
 'https://fbref.com/en/squads/d53c0b06/2022-2023/Lyon-Stats',
 'https://fbref.com/en/squads/d9676424/2022-2023/Clermont-Foot-Stats',
 'https://fbref.com/en/squads/132ebc33/2022-2023/Nice-Stats',
 'https://fbref.com/en/squads/d2c87802/2022-2023/Lorient-Stats',
 'https://fbref.com/en/squads/7fdd64e0/2022-2023/Reims-Stats',
 'https://fbref.com/en/squads/281b0e73/2022-2023/Montpellier-Stats',
 'https://fbref.com/en/squads/3f8c4b5f/2022-2023/Toulouse-Stats',
 'https://fbref.com/en/squads/fb08dbb3/2022-2023/Brest-Stats',
 'https://fbref.com/en/squads/c0d3eab4/2022-2023/Strasbourg-Stats',
 'https://fbre

##Bundesliga adatok

In [ ]:
# Az url-címen levő táblák caption-jeit nézi meg. Ha egyezést talál (match-el) az egyik tábla captionje a beírttal akkor leszedi annak az adatait.
players = pd.read_html(data.text, match="Standard Stats ")[0]

In [ ]:
# Ez eltüntette a fejléc felesleges elemeit, ezzel is javítva a táblázat kinézetét.
players.columns=players.columns.droplevel()

In [ ]:
players.head()

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,Ast,G+A,G-PK,G+A-PK,xG,xAG,xG+xAG,npxG,npxG+xAG,Matches
0,Gianluigi Donnarumma,it ITA,GK,23.0,38,38,3409.0,37.9,0.0,0.0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
1,Lionel Messi,ar ARG,"FW,MF",35.0,32,32,2837.0,31.5,16.0,16.0,...,0.51,1.02,0.51,1.02,0.49,0.43,0.92,0.49,0.92,Matches
2,Kylian Mbappé,fr FRA,FW,23.0,34,32,2818.0,31.3,29.0,5.0,...,0.16,1.09,0.83,0.99,0.84,0.23,1.07,0.71,0.94,Matches
3,Sergio Ramos,es ESP,DF,36.0,33,31,2704.0,30.0,2.0,1.0,...,0.03,0.10,0.07,0.10,0.06,0.01,0.06,0.06,0.06,Matches
4,Marquinhos,br BRA,DF,28.0,33,30,2717.0,30.2,2.0,0.0,...,0.00,0.07,0.07,0.07,0.07,0.02,0.09,0.07,0.09,Matches


In [ ]:
goalkeeping = pd.read_html(data.text, match="Goalkeeping ")[0]
goalkeeping.columns=goalkeeping.columns.droplevel()

In [ ]:
shooting = pd.read_html(data.text, match="Shooting ")[0]
shooting.columns=shooting.columns.droplevel()

In [ ]:
passing = pd.read_html(data.text, match="Passing ")[0]
passing.columns=passing.columns.droplevel()

In [ ]:
defensive_actions = pd.read_html(data.text, match="Defensive Actions ")[0]
defensive_actions.columns=defensive_actions.columns.droplevel()

In [ ]:
miscellaneous_stats = pd.read_html(data.text, match="Miscellaneous Stats ")[0]
miscellaneous_stats.columns=miscellaneous_stats.columns.droplevel()

In [ ]:
merged_df = players.merge(goalkeeping[['Player','GA','SoTA','Saves','Save%','CS','CS%']], on='Player', how='outer')
merged_df = players.merge(shooting[['Player','Sh','SoT','SoT%','G/Sh','Dist']], on='Player', how='outer')
merged_df = merged_df.merge(passing[['Player','Cmp','Att','Cmp%','TotDist','PrgDist','xA']], on='Player', how='outer')
merged_df = merged_df.merge(defensive_actions[['Player','Tkl','TklW','Tkl%','Int','Tkl+Int','Clr','Err']], on='Player', how='outer')
merged_df = merged_df.merge(miscellaneous_stats[['Player','Won','Lost','Won%']], on='Player', how='outer')

In [ ]:
merged_df

##Minden versenysorozat

In [ ]:
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/Paris-Saint-Germain-Stats-All-Competitions' in l]

In [ ]:
links

['/en/squads/e2d8892c/2022-2023/all_comps/Paris-Saint-Germain-Stats-All-Competitions',
 '/en/squads/e2d8892c/2022-2023/all_comps/Paris-Saint-Germain-Stats-All-Competitions',
 '/en/squads/e2d8892c/2022-2023/all_comps/Paris-Saint-Germain-Stats-All-Competitions',
 '/en/squads/e2d8892c/2022-2023/all_comps/Paris-Saint-Germain-Stats-All-Competitions']

In [ ]:
data = requests.get(f"https://fbref.com{links[0]}")

In [ ]:
all_comps = pd.read_html(data.text, match="Standard Stats")[0]

In [ ]:
all_comps.columns=all_comps.columns.droplevel()

In [ ]:
all_comps_goalkeeping = pd.read_html(data.text, match="Goalkeeping ")[0]
all_comps_goalkeeping.columns=all_comps_goalkeeping.columns.droplevel()

In [ ]:
all_comps_shooting = pd.read_html(data.text, match="Shooting ")[0]
all_comps_shooting.columns=all_comps_shooting.columns.droplevel()

In [ ]:
all_comps_passing = pd.read_html(data.text, match="Passing ")[0]
all_comps_passing.columns=all_comps_passing.columns.droplevel()

In [ ]:
all_comps_defensive_actions = pd.read_html(data.text, match="Defensive Actions ")[0]
all_comps_defensive_actions.columns=all_comps_defensive_actions.columns.droplevel()

In [ ]:
all_comps_miscellaneous_stats = pd.read_html(data.text, match="Miscellaneous Stats ")[0]
all_comps_miscellaneous_stats.columns=all_comps_miscellaneous_stats.columns.droplevel()

In [ ]:
all_comps_merged_df = all_comps.merge(all_comps_goalkeeping[['Player','GA','SoTA','Saves','Save%','CS','CS%']], on='Player', how='outer')
all_comps_merged_df = all_comps_merged_df.merge(all_comps_shooting[['Player','Sh','SoT','SoT%','G/Sh','Dist']], on='Player', how='outer')
all_comps_merged_df = all_comps_merged_df.merge(all_comps_passing[['Player','Cmp','Att','Cmp%','TotDist','PrgDist','xA']], on='Player', how='outer')
all_comps_merged_df = all_comps_merged_df.merge(all_comps_defensive_actions[['Player','Tkl','TklW','Tkl%','Int','Tkl+Int','Clr','Err']], on='Player', how='outer')
all_comps_merged_df = all_comps_merged_df.merge(all_comps_miscellaneous_stats[['Player','Won','Lost','Won%']], on='Player', how='outer')

In [ ]:
all_comps_merged_df

##Adatgyűjtés visszamenőleg valamint azok mergelése dupla for ciklussal, valamint a kapott adatok konkatanálása egy dataframe-mé

###Bundesliga

In [ ]:
import time
url_stats = "https://fbref.com/en/comps/13/2022-2023/2022-2023-Ligue-1-Stats"
statsBL = []
for year in range(2022, 2017, -1):
    url_stats = f"https://fbref.com/en/comps/13/{year}-{year+1}/" \
                  f"{year}-{year+1}-Ligue-1-Stats"
    data = requests.get(url_stats)
    soup = BeautifulSoup(data.text)
    BLstats_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in BLstats_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    url_stats = f"https://fbref.com/{previous_season}"

    for team_url in team_urls:
      teams = team_url.split("/")[-1].replace("-Stats","")
      team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
      competition= url_stats.split("/")[-1].split("-")[2].replace("-Stats","")
      season=f"{year}-{year+1}"

      data = requests.get(team_url)
      players = pd.read_html(data.text, match="Standard Stats ")[0]
      players.columns=players.columns.droplevel()
      goalkeeping = pd.read_html(data.text, match="Goalkeeping ")[0]
      goalkeeping.columns=goalkeeping.columns.droplevel()
      shooting = pd.read_html(data.text, match="Shooting ")[0]
      shooting.columns=shooting.columns.droplevel()
      passing = pd.read_html(data.text, match="Passing ")[0]
      passing.columns=passing.columns.droplevel()
      defensive_actions = pd.read_html(data.text, match="Defensive Actions ")[0]
      defensive_actions.columns=defensive_actions.columns.droplevel()
      miscellaneous_stats = pd.read_html(data.text, match="Miscellaneous Stats ")[0]
      miscellaneous_stats.columns=miscellaneous_stats.columns.droplevel()

      merged_df = players.merge(goalkeeping[['Player','GA','SoTA','Saves','Save%','CS','CS%']], on='Player', how='outer')
      merged_df = merged_df.merge(shooting[['Player','Sh','SoT','SoT%','G/Sh','Dist']], on='Player', how='outer')
      merged_df = merged_df.merge(passing[['Player','Cmp','Att','Cmp%','TotDist','PrgDist','xA']], on='Player', how='outer')
      merged_df = merged_df.merge(defensive_actions[['Player','Tkl','TklW','Tkl%','Int','Tkl+Int','Clr','Err']], on='Player', how='outer')
      merged_df = merged_df.merge(miscellaneous_stats[['Player','Won','Lost','Won%']], on='Player', how='outer')


      merged_df["Season"] = season
      merged_df["Team"] = team_name
      merged_df["Competition"]=competition

      statsBL.append(merged_df)
      time.sleep(1)

In [ ]:
BL=pd.concat(statsBL)

In [ ]:
BL

In [ ]:
BL["Competition"]="Ligue 1"
BL

In [ ]:
BL.to_csv("Ligue1.csv")

###All Competitions

In [ ]:
import time
url_stats = "https://fbref.com/en/comps/13/2022-2023/2022-2023-Ligue-1-Stats"
statsBL_AllComp = []
for year in range(2022, 2017, -1):
    url_stats = f"https://fbref.com/en/comps/13/{year}-{year+1}/" \
                  f"{year}-{year+1}-Ligue-1-Stats"
    data = requests.get(url_stats)
    soup = BeautifulSoup(data.text)
    BLstats_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in BLstats_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]

    previous_season = soup.select("a.prev")[0].get("href")
    url_stats = f"https://fbref.com/{previous_season}"

    for team_url in team_urls:
      teams = team_url.split("/")[-1].replace("-Stats","")
      team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")
      competition= url_stats.split("/")[-1].split("-")[2].replace("-Stats","")
      season=f"{year}-{year+1}"

      data = requests.get(team_url)
      soup = BeautifulSoup(data.text)
      links = soup.find_all('a')
      links = [l.get("href") for l in links]
      links = [l for l in links if l and '/all_comps/{}-Stats-All-Competitions'.format(teams) in l]
      data = requests.get(f"https://fbref.com{links[0]}")
      all_comps = pd.read_html(data.text, match="Standard Stats")[0]
      all_comps.columns=all_comps.columns.droplevel()
      all_comps_goalkeeping = pd.read_html(data.text, match="Goalkeeping ")[0]
      all_comps_goalkeeping.columns=all_comps_goalkeeping.columns.droplevel()
      all_comps_shooting = pd.read_html(data.text, match="Shooting ")[0]
      all_comps_shooting.columns=all_comps_shooting.columns.droplevel()
      all_comps_passing = pd.read_html(data.text, match="Passing ")[0]
      all_comps_passing.columns=all_comps_passing.columns.droplevel()
      all_comps_defensive_actions = pd.read_html(data.text, match="Defensive Actions ")[0]
      all_comps_defensive_actions.columns=all_comps_defensive_actions.columns.droplevel()
      all_comps_miscellaneous_stats = pd.read_html(data.text, match="Miscellaneous Stats ")[0]
      all_comps_miscellaneous_stats.columns=all_comps_miscellaneous_stats.columns.droplevel()

      all_comps_merged_df = all_comps.merge(all_comps_goalkeeping[['Player','GA','SoTA','Saves','Save%','CS','CS%']], on='Player', how='outer')
      all_comps_merged_df = all_comps_merged_df.merge(all_comps_shooting[['Player','Sh','SoT','SoT%','G/Sh','Dist']], on='Player', how='outer')
      all_comps_merged_df = all_comps_merged_df.merge(all_comps_passing[['Player','Cmp','Att','Cmp%','TotDist','PrgDist','xA']], on='Player', how='outer')
      all_comps_merged_df = all_comps_merged_df.merge(all_comps_defensive_actions[['Player','Tkl','TklW','Tkl%','Int','Tkl+Int','Clr','Err']], on='Player', how='outer')
      all_comps_merged_df = all_comps_merged_df.merge(all_comps_miscellaneous_stats[['Player','Won','Lost','Won%']], on='Player', how='outer')




      all_comps_merged_df["Season"] = season
      all_comps_merged_df["Team"] = team_name
      all_comps_merged_df["Competition"]=competition

      statsBL_AllComp.append(all_comps_merged_df)
      time.sleep(1)

In [ ]:
BLallcomp_df=pd.concat(statsBL_AllComp)

In [ ]:
BLallcomp_df

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,Int,Tkl+Int,Clr,Err,Won,Lost,Won%,Season,Team,Competition
0,Gianluigi Donnarumma,it ITA,GK,23,48,48,4309.0,47.9,0.0,0.0,...,0.0,1.0,19.0,5.0,7.0,1.0,87.5,2022-2023,Paris Saint Germain,Ligue
1,Sergio Ramos,es ESP,DF,36,45,42,3695.0,41.1,4.0,1.0,...,47.0,88.0,118.0,3.0,72.0,45.0,61.5,2022-2023,Paris Saint Germain,Ligue
2,Marquinhos,br BRA,DF,28,44,41,3640.0,40.4,2.0,0.0,...,28.0,64.0,103.0,2.0,49.0,35.0,58.3,2022-2023,Paris Saint Germain,Ligue
3,Lionel Messi,ar ARG,"FW,MF",35,41,41,3630.0,40.3,21.0,20.0,...,5.0,35.0,0.0,0.0,2.0,3.0,40.0,2022-2023,Paris Saint Germain,Ligue
4,Kylian Mbappé,fr FRA,FW,23,43,40,3559.0,39.5,41.0,9.0,...,5.0,15.0,9.0,0.0,9.0,7.0,56.3,2022-2023,Paris Saint Germain,Ligue
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,Louis Carnot,fr FRA,MF,17,1,0,8.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2018-2019,Guingamp,Ligue
30,Denis Petrić,rs SRB,GK,30,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue
31,Yohan Bilingi,fr FRA,DF,19,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue
32,Mehdi Boudjemaa,dz ALG,MF,20,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue


In [ ]:
BLallcomp_df["Competition"]="Ligue 1"
BLallcomp_df

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,Int,Tkl+Int,Clr,Err,Won,Lost,Won%,Season,Team,Competition
0,Gianluigi Donnarumma,it ITA,GK,23,48,48,4309.0,47.9,0.0,0.0,...,0.0,1.0,19.0,5.0,7.0,1.0,87.5,2022-2023,Paris Saint Germain,Ligue 1
1,Sergio Ramos,es ESP,DF,36,45,42,3695.0,41.1,4.0,1.0,...,47.0,88.0,118.0,3.0,72.0,45.0,61.5,2022-2023,Paris Saint Germain,Ligue 1
2,Marquinhos,br BRA,DF,28,44,41,3640.0,40.4,2.0,0.0,...,28.0,64.0,103.0,2.0,49.0,35.0,58.3,2022-2023,Paris Saint Germain,Ligue 1
3,Lionel Messi,ar ARG,"FW,MF",35,41,41,3630.0,40.3,21.0,20.0,...,5.0,35.0,0.0,0.0,2.0,3.0,40.0,2022-2023,Paris Saint Germain,Ligue 1
4,Kylian Mbappé,fr FRA,FW,23,43,40,3559.0,39.5,41.0,9.0,...,5.0,15.0,9.0,0.0,9.0,7.0,56.3,2022-2023,Paris Saint Germain,Ligue 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,Louis Carnot,fr FRA,MF,17,1,0,8.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2018-2019,Guingamp,Ligue 1
30,Denis Petrić,rs SRB,GK,30,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue 1
31,Yohan Bilingi,fr FRA,DF,19,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue 1
32,Mehdi Boudjemaa,dz ALG,MF,20,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue 1


In [ ]:
BLallcomp_df.to_csv("Ligue1_All_Comp.csv")

# Adattisztítás


*   A webscrapping során kapott .csv file-ok beolvasása egy-egy új dataframebe
*   Felesleges oszlopok illetve sorok eltávolítása
*   A lecsupaszított dataframe-ek kiimportálása egy-egy új .csv file-ba



## Bundesliga adatok

In [ ]:
BL=pd.read_csv("PL.csv", index_col=0)

In [ ]:
del BL["npxG+xAG"]
del BL["Cmp.1"]
del BL["Cmp.2"]
del BL["Cmp.3"]
del BL["Att.1"]
del BL["Att.2"]
del BL["Att.3"]
del BL["Cmp%.1"]
del BL["Cmp%.2"]
del BL["Cmp%.3"]
del BL["PrgC"]
del BL["PrgP"]
del BL["PrgR"]
del BL["Gls.1"]
del BL["Ast.1"]
del BL["G+A.1"]
del BL["G-PK.1"]
del BL["G+A-PK"]
del BL["xG.1"]
del BL["xAG.1"]
del BL["npxG+xAG.1"]
del BL["Matches"]
del BL["xG+xAG"]
del BL['npxG.1']

In [ ]:
BL.rename(columns={"PK" : "Penalty_Kicks_Made","Save%.1" : "PK_Save%","Tkl.1" : "Challenges_Tkl" }, inplace=True)


In [ ]:
BL=BL[BL["Player"]!="Squad Total"]

In [ ]:
BL=BL[BL["Player"]!="Opponent Total"]

In [ ]:
BL.to_csv("PL_vegleges.csv")

## Minden versenysorozat adata

In [ ]:
BL_All=pd.read_csv("Ligue1_All_Comp.csv", index_col=0)

In [ ]:
del BL_All["npxG+xAG"]
del BL_All["Cmp.1"]
del BL_All["Cmp.2"]
del BL_All["Cmp.3"]
del BL_All["Att.1"]
del BL_All["Att.2"]
del BL_All["Att.3"]
del BL_All["Cmp%.1"]
del BL_All["Cmp%.2"]
del BL_All["Cmp%.3"]
del BL_All["PrgC"]
del BL_All["PrgP"]
del BL_All["PrgR"]
del BL_All["Gls.1"]
del BL_All["Ast.1"]
del BL_All["G+A.1"]
del BL_All["G-PK.1"]
del BL_All["G+A-PK"]
del BL_All["xG.1"]
del BL_All["xAG.1"]
del BL_All["npxG+xAG.1"]
del BL_All["Matches"]
del BL_All["xG+xAG"]

In [ ]:
BL_All.rename(columns={"PK" : "Penalty_Kicks_Made","Save%.1" : "PK_Save%","Tkl.1" : "Challenges_Tkl" }, inplace=True)

In [ ]:
BL_All[BL_All["Player"]=="Opponent Total"]

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,Int,Tkl+Int,Clr,Err,Won,Lost,Won%,Season,Team,Competition


In [ ]:
BL_All[BL_All["Player"]=="Squad Total"]

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,Int,Tkl+Int,Clr,Err,Won,Lost,Won%,Season,Team,Competition


In [ ]:
BL_All=BL_All[BL_All["Player"]!="Opponent Total"]
BL_All=BL_All[BL_All["Player"]!="Squad Total"]

In [ ]:
BL_All

,Player,Nation,Pos,Age,MP,Starts,Min,90s,Gls,Ast,...,Int,Tkl+Int,Clr,Err,Won,Lost,Won%,Season,Team,Competition
0,Gianluigi Donnarumma,it ITA,GK,23,48,48,4309.0,47.9,0.0,0.0,...,0.0,1.0,19.0,5.0,7.0,1.0,87.5,2022-2023,Paris Saint Germain,Ligue 1
1,Sergio Ramos,es ESP,DF,36,45,42,3695.0,41.1,4.0,1.0,...,47.0,88.0,118.0,3.0,72.0,45.0,61.5,2022-2023,Paris Saint Germain,Ligue 1
2,Marquinhos,br BRA,DF,28,44,41,3640.0,40.4,2.0,0.0,...,28.0,64.0,103.0,2.0,49.0,35.0,58.3,2022-2023,Paris Saint Germain,Ligue 1
3,Lionel Messi,ar ARG,"FW,MF",35,41,41,3630.0,40.3,21.0,20.0,...,5.0,35.0,0.0,0.0,2.0,3.0,40.0,2022-2023,Paris Saint Germain,Ligue 1
4,Kylian Mbappé,fr FRA,FW,23,43,40,3559.0,39.5,41.0,9.0,...,5.0,15.0,9.0,0.0,9.0,7.0,56.3,2022-2023,Paris Saint Germain,Ligue 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,Louis Carnot,fr FRA,MF,17,1,0,8.0,0.1,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,2018-2019,Guingamp,Ligue 1
30,Denis Petrić,rs SRB,GK,30,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue 1
31,Yohan Bilingi,fr FRA,DF,19,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue 1
32,Mehdi Boudjemaa,dz ALG,MF,20,0,0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2018-2019,Guingamp,Ligue 1


In [ ]:
BL_All.to_csv("Ligue1_All_vegleges.csv")

#Bajnoki tabellák
##Meg szeretném vizsgálni hogy egyes csapatok meglepetésszerű mélyrepülése vagy szárnyalása mennyiben köszönhető egyes kulcsjátékosainak a kiemelkedő teljesítményéhez és milyen formában valósul ez meg.

In [ ]:
url_stats = "https://fbref.com/en/comps/9/2022-2023/2022-2023-Premier-League-Stats"

In [ ]:
data = requests.get(url_stats)

In [ ]:
soup=BeautifulSoup(data.text,"html.parser")

In [ ]:
stats_table = soup.select('table.stats_table')[0]

In [ ]:
standings= pd.read_html(str(stats_table))[0]

In [ ]:
standings

In [ ]:
import time
url_stats = "https://fbref.com/en/comps/20/2022-2023/2022-2023-Bundesliga-Stats"
standings_table = []
for year in range(2022, 2017, -1):
    url_stats = f"https://fbref.com/en/comps/20/{year}-{year+1}/" \
                  f"{year}-{year+1}-Bundesliga-Stats"
    data = requests.get(url_stats)
    soup=BeautifulSoup(data.text,"html.parser")
    stats_table = soup.select('table.stats_table')[0]
    standings= pd.read_html(str(stats_table))[0]
    previous_season = soup.select("a.prev")[0].get("href")
    url_stats = f"https://fbref.com/{previous_season}"
    competition= url_stats.split("/")[-1].split("-")[2].replace("-Stats","")
    season=f"{year}-{year+1}"

    standings["Season"] = season
    standings["Competition"]=competition

    standings_table.append(standings)
    time.sleep(1)

In [ ]:
df=pd.concat(standings_table)

In [ ]:
df["Competition"]="Seria A"

In [ ]:
del df["Pts/MP"]
del df["xG"]
del df["xGA"]
del df["xGD"]
del df["xGD/90"]
del df["Attendance"]
del df["Top Team Scorer"]
del df["Goalkeeper"]
del df["Notes"]

In [ ]:
df

,Rk,Squad,MP,W,D,L,GF,GA,GD,Pts,Season,Competition
0,1,Bayern Munich,34,21,8,5,92,38,54,71,2022-2023,Bundesliga
1,2,Dortmund,34,22,5,7,83,44,39,71,2022-2023,Bundesliga
2,3,RB Leipzig,34,20,6,8,64,41,23,66,2022-2023,Bundesliga
3,4,Union Berlin,34,18,8,8,51,38,13,62,2022-2023,Bundesliga
4,5,Freiburg,34,17,8,9,51,44,7,59,2022-2023,Bundesliga
...,...,...,...,...,...,...,...,...,...,...,...,...
13,14,Schalke 04,34,8,9,17,37,55,-18,33,2018-2019,Bundesliga
14,15,Augsburg,34,8,8,18,51,71,-20,32,2018-2019,Bundesliga
15,16,Stuttgart,34,7,7,20,32,70,-38,28,2018-2019,Bundesliga
16,17,Hannover 96,34,5,6,23,31,71,-40,21,2018-2019,Bundesliga


In [ ]:
df.to_csv("Bundesliga.csv")

#Aranylabda győztesek
A futball világ legnagyobb presztizsű egyéni díja az Aranylabda, amelyet az adott szezon legjobb játékosa nyer el minden évben. 2018-2019 ÉS 2022-2023 között 4-szer hírdettek győztest. Meg szeretném jósolni, hogy nyerni a 2023-2024-es szezon Aranylabdáját.

##2023

In [ ]:
url_stats = "https://en.wikipedia.org/wiki/2023_Ballon_d%27Or"

In [ ]:
data = requests.get(url_stats)

In [ ]:
soup=BeautifulSoup(data.text,"html.parser")

In [ ]:
ranking_table = soup.select('table.wikitable')[0]

In [ ]:
ranking2023= pd.read_html(str(ranking_table))[0]

In [ ]:
ranking2023["Points%"]=ranking2023['Points']/ranking2023['Points'].sum()

In [ ]:
ranking2023

,Rank,Player,Nationality,Position,Club,Points,Points%
0,1,Lionel Messi,Argentina,Forward,Inter Miami[a],462,0.313859
1,2,Erling Haaland,Norway,Forward,Manchester City,357,0.242527
2,3,Kylian Mbappé,France,Forward,Paris Saint-Germain,270,0.183424
3,4,Kevin De Bruyne,Belgium,Midfielder,Manchester City,100,0.067935
4,5,Rodri,Spain,Midfielder,Manchester City,57,0.038723
5,6,Vinícius Júnior,Brazil,Forward,Real Madrid,49,0.033288
6,7,Julián Álvarez,Argentina,Forward,Manchester City,28,0.019022
7,8,Victor Osimhen,Nigeria,Forward,Napoli,24,0.016304
8,9,Bernardo Silva,Portugal,Midfielder,Manchester City,20,0.013587
9,10,Luka Modrić,Croatia,Midfielder,Real Madrid,19,0.012908


##2022

In [ ]:
url_stats = "https://en.wikipedia.org/wiki/2022_Ballon_d%27Or"

In [ ]:
data = requests.get(url_stats)

In [ ]:
soup=BeautifulSoup(data.text,"html.parser")

In [ ]:
ranking_table = soup.select('table.wikitable')[0]

In [ ]:
ranking2022= pd.read_html(str(ranking_table))[0]

In [ ]:
ranking2022["Points%"]=ranking2022['Points']/ranking2022['Points'].sum()

In [ ]:
ranking2022

,Rank,Player,Nationality,Position,Club,Points,Points%
0,1,Karim Benzema,France,Forward,Real Madrid,549,0.368952
1,2,Sadio Mané,Senegal,Forward,Liverpool,193,0.129704
2,3,Kevin De Bruyne,Belgium,Midfielder,Manchester City,175,0.117608
3,4,Robert Lewandowski,Poland,Forward,Bayern Munich,170,0.114247
4,5,Mohamed Salah,Egypt,Forward,Liverpool,116,0.077957
5,6,Kylian Mbappé,France,Forward,Paris Saint-Germain,85,0.057124
6,7,Thibaut Courtois,Belgium,Goalkeeper,Real Madrid,82,0.055108
7,8,Vinícius Júnior,Brazil,Midfielder,Real Madrid,61,0.040995
8,9,Luka Modrić,Croatia,Midfielder,Real Madrid,20,0.013441
9,10,Erling Haaland,Norway,Forward,Borussia Dortmund,18,0.012097


##2021

In [ ]:
url_stats = "https://en.wikipedia.org/wiki/2021_Ballon_d%27Or"

In [ ]:
data = requests.get(url_stats)

In [ ]:
soup=BeautifulSoup(data.text,"html.parser")

In [ ]:
ranking_table = soup.select('table.wikitable')[0]

In [ ]:
ranking2021= pd.read_html(str(ranking_table))[0]

In [ ]:
ranking2021["Points%"]=ranking2021['Points']/ranking2021['Points'].sum()

In [ ]:
ranking2021

,Rank,Player,Club(s),Points,Points%
0,1,Lionel Messi,Barcelona Paris Saint-Germain,613,0.225368
1,2,Robert Lewandowski,Bayern Munich,580,0.213235
2,3,Jorginho,Chelsea,460,0.169118
3,4,Karim Benzema,Real Madrid,239,0.087868
4,5,N'Golo Kanté,Chelsea,186,0.068382
5,6,Cristiano Ronaldo,Juventus Manchester United,178,0.065441
6,7,Mohamed Salah,Liverpool,121,0.044485
7,8,Kevin De Bruyne,Manchester City,73,0.026838
8,9,Kylian Mbappé,Paris Saint-Germain,58,0.021324
9,10,Gianluigi Donnarumma,Milan Paris Saint-Germain,36,0.013235


##2019

In [ ]:
url_stats = "https://en.wikipedia.org/wiki/2019_Ballon_d%27Or"

In [ ]:
data = requests.get(url_stats)

In [ ]:
soup=BeautifulSoup(data.text,"html.parser")

In [ ]:
ranking_table = soup.select('table.wikitable')[0]

In [ ]:
ranking2019= pd.read_html(str(ranking_table))[0]

In [ ]:
ranking2019["Points%"]=ranking2019['Points']/ranking2019['Points'].sum()

In [ ]:
ranking2019

,Rank,Player,Club(s),Points,Points%
0,1,Lionel Messi,Barcelona,686,0.243608
1,2,Virgil van Dijk,Liverpool,679,0.241122
2,3,Cristiano Ronaldo,Juventus,476,0.169034
3,4,Sadio Mané,Liverpool,347,0.123224
4,5,Mohamed Salah,Liverpool,178,0.063210
5,6,Kylian Mbappé,Paris Saint-Germain,89,0.031605
6,7,Alisson,Liverpool,67,0.023793
7,8,Robert Lewandowski,Bayern Munich,44,0.015625
8,9,Bernardo Silva,Manchester City,41,0.014560
9,10,Riyad Mahrez,Manchester City,33,0.011719


##Dataframek formázása és egyesítése

In [ ]:
top10_2023=ranking2023.nlargest(10,'Points')
top10_2023["Season"]="2022-2023"
del top10_2023['Nationality']
del top10_2023['Position']
top10_2023["Club"]=top10_2023["Club"].str.replace("Inter Miami[a]","Paris Saint-Germain", regex=False)
top10_2023.rename(columns={"Club" : "Team"}, inplace=True)
top10_2023

,Rank,Player,Team,Points,Points%,Season
0,1,Lionel Messi,Paris Saint-Germain,462,0.313859,2022-2023
1,2,Erling Haaland,Manchester City,357,0.242527,2022-2023
2,3,Kylian Mbappé,Paris Saint-Germain,270,0.183424,2022-2023
3,4,Kevin De Bruyne,Manchester City,100,0.067935,2022-2023
4,5,Rodri,Manchester City,57,0.038723,2022-2023
5,6,Vinícius Júnior,Real Madrid,49,0.033288,2022-2023
6,7,Julián Álvarez,Manchester City,28,0.019022,2022-2023
7,8,Victor Osimhen,Napoli,24,0.016304,2022-2023
8,9,Bernardo Silva,Manchester City,20,0.013587,2022-2023
9,10,Luka Modrić,Real Madrid,19,0.012908,2022-2023


In [ ]:
top10_2022=ranking2022.nlargest(10,'Points')
top10_2022["Season"]="2021-2022"
del top10_2022['Nationality']
del top10_2022['Position']
top10_2022.rename(columns={"Club" : "Team"}, inplace=True)
top10_2022

,Rank,Player,Team,Points,Points%,Season
0,1,Karim Benzema,Real Madrid,549,0.368952,2021-2022
1,2,Sadio Mané,Liverpool,193,0.129704,2021-2022
2,3,Kevin De Bruyne,Manchester City,175,0.117608,2021-2022
3,4,Robert Lewandowski,Bayern Munich,170,0.114247,2021-2022
4,5,Mohamed Salah,Liverpool,116,0.077957,2021-2022
5,6,Kylian Mbappé,Paris Saint-Germain,85,0.057124,2021-2022
6,7,Thibaut Courtois,Real Madrid,82,0.055108,2021-2022
7,8,Vinícius Júnior,Real Madrid,61,0.040995,2021-2022
8,9,Luka Modrić,Real Madrid,20,0.013441,2021-2022
9,10,Erling Haaland,Borussia Dortmund,18,0.012097,2021-2022


In [ ]:
top10_2021=ranking2021.nlargest(10,'Points')
top10_2021["Season"]="2020-2021"
top10_2021.rename(columns={"Club(s)" : "Team"}, inplace=True)

top10_2021

,Rank,Player,Team,Points,Points%,Season
0,1,Lionel Messi,Barcelona Paris Saint-Germain,613,0.225368,2020-2021
1,2,Robert Lewandowski,Bayern Munich,580,0.213235,2020-2021
2,3,Jorginho,Chelsea,460,0.169118,2020-2021
3,4,Karim Benzema,Real Madrid,239,0.087868,2020-2021
4,5,N'Golo Kanté,Chelsea,186,0.068382,2020-2021
5,6,Cristiano Ronaldo,Juventus Manchester United,178,0.065441,2020-2021
6,7,Mohamed Salah,Liverpool,121,0.044485,2020-2021
7,8,Kevin De Bruyne,Manchester City,73,0.026838,2020-2021
8,9,Kylian Mbappé,Paris Saint-Germain,58,0.021324,2020-2021
9,10,Gianluigi Donnarumma,Milan Paris Saint-Germain,36,0.013235,2020-2021


In [ ]:
top10_2019=ranking2019.nlargest(10,'Points')
top10_2019["Season"]="2018-2019"
top10_2019.rename(columns={"Club(s)" : "Team"}, inplace=True)
top10_2019

,Rank,Player,Team,Points,Points%,Season
0,1,Lionel Messi,Barcelona,686,0.243608,2018-2019
1,2,Virgil van Dijk,Liverpool,679,0.241122,2018-2019
2,3,Cristiano Ronaldo,Juventus,476,0.169034,2018-2019
3,4,Sadio Mané,Liverpool,347,0.123224,2018-2019
4,5,Mohamed Salah,Liverpool,178,0.063210,2018-2019
5,6,Kylian Mbappé,Paris Saint-Germain,89,0.031605,2018-2019
6,7,Alisson,Liverpool,67,0.023793,2018-2019
7,8,Robert Lewandowski,Bayern Munich,44,0.015625,2018-2019
8,9,Bernardo Silva,Manchester City,41,0.014560,2018-2019
9,10,Riyad Mahrez,Manchester City,33,0.011719,2018-2019


In [ ]:
bd_df = pd.concat([top10_2023, top10_2022, top10_2021, top10_2019], ignore_index=True)
bd_df

NameError: name 'pd' is not defined

In [ ]:
bd_df.to_csv("Aranylabda.csv")